In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
from keras.models import Model
from IPython.display import display
from IPython.display import Image as _Imgdis
from keras.preprocessing.image import array_to_img, img_to_array, load_img

image_path = '/content/gdrive/MyDrive/pytest_img/opencv/'
save_path = '/content/gdrive/MyDrive/pytest_img/_generated_images'

# 원본 이미지와 참조 이미지 경로 설정
base_image_path = image_path+'seoul.png'
style_reference_image_path = image_path+'starnight.png'

In [3]:
import os
# 기본 저장 경로 밑에 neural_style 이라는 폴더를 만든다
if not os.path.exists(os.path.join(save_path, "neural_style/")):
  os.makedirs(os.path.join(os.path.join(save_path, "neural_style/")))

In [4]:
# 원본 이미지와 참조 이미지의 사이즈가 비슷해야 잘 된다
# 두 이미지를 같은 크기로 하였을 때 출력에 무리가 없는지 확인한다
img_height = 400 # 이미지의 높이
img_width = 600 # 이미지의 너비
display(_Imgdis(filename=base_image_path, height=img_height, width=img_width))
display(_Imgdis(filename=style_reference_image_path, height=img_height, width=img_width))

Output hidden; open in https://colab.research.google.com to view.

In [5]:
#이미지를 읽고, 전처리하는 함수를 정의한다
import numpy as np
import tensorflow as tf
def preprocess_image(image_path):
  img = load_img(image_path, target_size=(img_height, img_width)) # 같은 사이즈로 맞춘다
  img = img_to_array(img)
  img = np.expand_dims(img, axis=0) # 맨 앞에 배치 차원을 추가한다
  img = tf.keras.applications.vgg19.preprocess_input(img)
  return img

In [6]:
def deprocess_image(img):
  img = img.reshape((img_height, img_width, 3)) # 이미지를 위에서 정의한 사이즈로 변환
  # VGG19.preprocess_input()에서는 전처리를 수행하면서 RGB 채널에서 아래 세 값을 빼주므로 다시 더하여 원래 색상으로 복원한다.
  # 아래 세 값은 ImageNet의 평균 필셀 값
  img[:, :, 0] += 103.939
  img[:, :, 1] += 116.779
  img[:, :, 2] += 123.68
  img = img[:, :, ::-1] # 채널의 순서를 VGG19가 학습된 RGB → BGR로 변경한다
  img = np.clip(img, 0, 255).astype("uint8") # 값을 0~255 사이로 제한한다
  return img

In [7]:
from tensorflow.keras.applications.vgg19 import VGG19
model = VGG19(weights="imagenet", include_top=False)

80134624/80134624 [==============================] - 4s 0us/step


In [8]:
# 모델의 레이어들을 그 출력 결과와 함께 가져온다
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

# 모델에 입력을 넣으면 모델의 모든 레이어 출력을 포함하는 딕셔너리를 반환하는 모델 생성
# 일반적으로 Functional API에서 outputs에는 최종 출력층을 넣지만,
# 여기에서는 필요한 레이어의 출력값을 쉽게 가져오도록 모든 레이어의 출력을 반환하게 한다
feature_extractor = Model(inputs=model.inputs, outputs=outputs_dict)

In [9]:
outputs_dict

{'input_1': <KerasTensor: shape=(None, None, None, 3) dtype=float32 (created by layer 'input_1')>,
 'block1_conv1': <KerasTensor: shape=(None, None, None, 64) dtype=float32 (created by layer 'block1_conv1')>,
 'block1_conv2': <KerasTensor: shape=(None, None, None, 64) dtype=float32 (created by layer 'block1_conv2')>,
 'block1_pool': <KerasTensor: shape=(None, None, None, 64) dtype=float32 (created by layer 'block1_pool')>,
 'block2_conv1': <KerasTensor: shape=(None, None, None, 128) dtype=float32 (created by layer 'block2_conv1')>,
 'block2_conv2': <KerasTensor: shape=(None, None, None, 128) dtype=float32 (created by layer 'block2_conv2')>,
 'block2_pool': <KerasTensor: shape=(None, None, None, 128) dtype=float32 (created by layer 'block2_pool')>,
 'block3_conv1': <KerasTensor: shape=(None, None, None, 256) dtype=float32 (created by layer 'block3_conv1')>,
 'block3_conv2': <KerasTensor: shape=(None, None, None, 256) dtype=float32 (created by layer 'block3_conv2')>,
 'block3_conv3': <Ke

In [10]:
# 콘텐츠 즉, 원본 이미지의 기본 형상에 대한 손실함수 정의
# base_img는 원본 이미지를,
# combination_img는 스타일이 적용된 후의 조합 이미지로 둘의 차이를 손실값으로 정의한다
# 변환된 값이 원본 이미지와 지나치게 차이가 나지 않도록 조절하는 데 사용된다
# 너무 같아도 안되므로, 스타일 손실과의 사이에서 적절한 밸런스를 찾는다
def content_loss(base_img, combination_img):
  return tf.reduce_sum(tf.square(combination_img - base_img))

In [11]:
def gram_matrix(x):
  x = tf.transpose(x, (2, 0, 1)) # (높이, 너비, 채널) → (채널, 높이, 너비) 순서로 바꾼다
  features = tf.reshape(x, (tf.shape(x)[0], -1)) # 각 채널을 행으로, 나머지는 -1로 (높이x너비) 열로 삼아 2D 행렬 구성
  gram = tf.matmul(features, tf.transpose(features)) # 만들어진 2D features를 그 전치와 행렬곱하여 그람 행렬을 만든다

  return gram

In [12]:
# 스타일, 즉 참조 이미지가 갖는 스타일에 대한 손실함수를 정의
# 참조 이미지와 스타일이 적용된 후의 이미지가 얼마나 차이나는지를 손실값으로 정의한다
# 스타일은 개별 픽셀의 차이보다는 채널 사이의 상호관계 차이로 보는 것이 더 타당하기 때문
def style_loss(style_img, combination_img):
  S = gram_matrix(style_img) # 스타일 이미지의 그람 행렬 계산
  C = gram_matrix(combination_img) # 스타일이 적용된 후의 이미지인 combination_img의 그람 행렬 계산
  channels = 3
  size = img_height * img_width
  return tf.reduce_sum(tf.square(S-C)) / (4.0 * (channels ** 2) * (size ** 2))

In [13]:
def total_variation_loss(x):
  a = tf.square(x[:, : img_height - 1, : img_width - 1, :] - x[:, 1:, : img_width - 1, :])
  b = tf.square(x[:, : img_height - 1, : img_width - 1, :] - x[:, : img_height - 1, 1:, :])
  return tf.reduce_sum(tf.pow(a+b, 1.25))

In [14]:
# 스타일을 추출하기 위해 사용할 네트워크 층
style_layer_names = ["block1_conv1", "block2_conv1", "block3_conv1", "block4_conv1", "block5_conv1"]

# 콘텐츠(원본 이미지)에서 기본 형상 특징을 추출하기 위해 사용할 네트워크 층
content_layer_name = "block5_conv2" # 콘텐츠 손실에 사용할 층

# 스타일을 얼마나 원본 이미지에 반영할 것인지를 결정하는 가중치 설정
total_variation_weight = 1e-6 # 총 변동 손실의 기여 가중치 (이미지를 얼마나 매끄럽게 표현할 것인가)
style_weight = 1e-6 # 스타일 손실의 기여 가중치 (전체 이미지에서 반영할 스타일의 비중)
content_weight = 2.5e-8 # 콘텐츠 손실의 기여 가중치 (전체 이미지에서 반영할 콘텐츠의 비중)

In [16]:
# 조합된 이미지, 원본 이미지, 참조 이미지를 입력으로 받아 그들의 차이를 토대로 최종 손실을 계산한다
def compute_loss(combination_image, base_image, style_reference_image):
  # 원본 이미지, 참조 이미지, 조합된 이미지를 배치 차원으로 결합한다 (3, 높이, 너비, 채널)
  input_tensor = tf.concat([base_image, style_reference_image, combination_image], axis=0)
  features = feature_extractor(input_tensor) # 입력 데이터를 모델을 사용하여 처리한 결과로부터 특징을 추출
  loss = tf.zeros(shape=()) # 손실을 0으로 초기화
  layer_features = features[content_layer_name] # 콘텐츠 손실에 사용할 층의 특징을 추출
  base_image_features = layer_features[0, :, :, :] # 원본 이미지의 특징을 추출
  combination_features = layer_features[2, :, :, :] # 조합 이미지의 특징을 추출
  loss += content_weight * content_loss(base_image_features, combination_features) # 콘텐츠 손실을 계산하고, 이를 콘텐츠 손실 가중치와 곱한 뒤, 전체 손실에 추가한다

  for layer_name in style_layer_names: # 스타일 추출에 사용할 층
    layer_features = features[layer_name] # 스타일 손실에 사용할 층의 특징을 추출
    style_reference_features = layer_features[1, :, :, :] # 스타일 이미지의 특징을 추출
    combination_features = layer_features[2, :, :, :] # 조합 이미지의 특징을 추출
    style_loss_value = style_loss(style_reference_features, combination_features) # 각 층의 스타일 손실 계산
    loss += (style_weight / len(style_layer_names)) * style_loss_value # 스타일 손실을 (스타일 손실 가중치/층 수)와 곱한 뒤, 전체 손실에 추가한다 (층의 모든 계산 결과를 누적)
  loss += total_variation_weight * total_variation_loss(combination_image) # 이미지의 매끄러움에 관계되는 조합 이미지의 총 변동 손실을 계산하고, 이를 총 변동 손실 가중치와 곱한 뒤, 전체 손실에 추가한다
  return loss

In [17]:
def compute_loss_and_grads(combination_image, base_image, style_reference_image):
  with tf.GradientTape() as tape: # 그라디언트 계산
    # 조합 이미지, 원본 이미지, 참조 이미지를 사용하여 최종 손실값을 계산한다
    loss = compute_loss(combination_image, base_image, style_reference_image)

  # 최종 손실값에 대한 그라디언트를 계산하여 손실을 최소화하는 방향과 크기를 파악한다
  grads = tape.gradient(loss, combination_image)
  return loss, grads

In [18]:
optimizer = tf.keras.optimizers.SGD(
  tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=100.0, decay_steps=100, decay_rate=0.90
  )
)

In [19]:
base_image = preprocess_image(base_image_path)
style_reference_image = preprocess_image(style_reference_image_path)
combination_image = tf.Variable(preprocess_image(base_image_path))# tf.GradientTape()을 사용할 수 있도록 tf.Variable()로 변경 가능한 텐서로 만든다

In [22]:
from tensorflow import keras
iterations = 4000
for i in range(1, iterations + 1):
  loss, grads = compute_loss_and_grads(combination_image, base_image, style_reference_image)
  optimizer.apply_gradients([(grads, combination_image)]) # 옵티마이저를 이용하여 grads가 적은 방향으로 조합 이미지 업데이트
  if i % 100 == 0:
    print(f"{i}번째 반복: loss={loss:.2f}")
    img = deprocess_image(combination_image.numpy()) # 이미지 복원 함수로 출력할 수 있는 상태를 만든다
    fname = f"combination_image_at_iteration_{i}.png" # 파일 이름 설정
    keras.utils.save_img(os.path.join(save_path, "neural_style/", fname), img) # 100번째마다 이미지 저장

100번째 반복: loss=12426.99
200번째 반복: loss=9811.43
300번째 반복: loss=8782.51
400번째 반복: loss=8215.18
500번째 반복: loss=7848.68
600번째 반복: loss=7589.55
700번째 반복: loss=7395.44
800번째 반복: loss=7244.43
900번째 반복: loss=7123.60
1000번째 반복: loss=7024.71
1100번째 반복: loss=6942.58
1200번째 반복: loss=6873.45
1300번째 반복: loss=6814.52
1400번째 반복: loss=6763.96
1500번째 반복: loss=6720.33
1600번째 반복: loss=6682.49
1700번째 반복: loss=6649.49
1800번째 반복: loss=6620.63
1900번째 반복: loss=6595.28
2000번째 반복: loss=6573.00
2100번째 반복: loss=6553.32
2200번째 반복: loss=6535.92
2300번째 반복: loss=6520.52
2400번째 반복: loss=6506.85
2500번째 반복: loss=6494.70
2600번째 반복: loss=6483.89
2700번째 반복: loss=6474.25
2800번째 반복: loss=6465.65
2900번째 반복: loss=6457.95
3000번째 반복: loss=6451.08
3100번째 반복: loss=6444.92
3200번째 반복: loss=6439.42
3300번째 반복: loss=6434.49
3400번째 반복: loss=6430.08
3500번째 반복: loss=6426.12
3600번째 반복: loss=6422.58
3700번째 반복: loss=6419.40
3800번째 반복: loss=6416.54
3900번째 반복: loss=6413.98
4000번째 반복: loss=6411.68
